In [ ]:
import numpy as np
import plotly.express as px
import random
import pandas as pd
import plotly.io as pio
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import plotly.graph_objects as go


np.random.seed(46)
pio.renderers.default = "colab"

# 1.  Для синтетических данных

## Генерация данных

### Линейные

In [ ]:
X = np.linspace(0, 6)
y = np.linspace(0, 3)

noise_factor = 0.2

def noise(k):
   return k+((random.random()*2)-1)*noise_factor

X_1 = np.vectorize(noise)(X)
y_1 = np.vectorize(noise)(y)

In [ ]:
px.scatter(x=X_1, y=y_1)

Сделали обычные линейные данные с шумом

### Полиномиальные

In [ ]:
X_2 = 2 - 3 * np.random.normal(0, 1, 100)
y_2 = X_2 - 2 * (X_2 ** 2) + 0.5 * (X_2 ** 3) + np.random.normal(-3, 3, 100)

In [ ]:
px.scatter(x=X_2, y=y_2)

Тут данные без особого шума 

### Полиномиальные с большим шумом

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def func(x, c): 
    return sum([c[i] * x**i for i in range(len(c))])

num_data_points = 300
degree = 3

X_3 = np.random.uniform(-10, 10, num_data_points)
c = [np.random.uniform(-3, 3) for _ in range(degree + 1)]
y_3 = np.array( [func(X_3[i], c) for i in range(num_data_points)] )

mu, sigma = 0, 100
noise = np.random.normal(mu, sigma, num_data_points)

y_3 = np.add(y_3, noise)

In [ ]:
px.scatter(x=X_3, y=y_3)

Здесь данные уже с большим шумом, сделали даные по формуле, далее добавили нормально распределённый шум, чтобы оценки минимальных квадратов были оценками максимального правдоподобия

## Train test split

In [ ]:
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1.reshape(-1, 1), y_1.reshape(-1, 1), test_size=0.2)

In [ ]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2.reshape(-1, 1), y_2.reshape(-1, 1), test_size=0.2)

In [ ]:
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X_3.reshape(-1, 1), y_3.reshape(-1, 1), test_size=0.2)

## Проверка моделей

## 1 датасет

### Линейная регрессия

In [ ]:
model_lr_1 = LinearRegression().fit(X_train_1, y_train_1)

y_pred_lr_1 = model_lr_1.predict(X_test_1)

print(f'R2 score: {r2_score(y_test_1, y_pred_lr_1)}')
print(f'MAE: {mean_absolute_error(y_test_1, y_pred_lr_1)}')
print(f'MSE: {mean_squared_error(y_test_1, y_pred_lr_1)}')

R2 score: 0.9720382826771956
MAE: 0.10434052507194762
MSE: 0.014797672285965745


Логика оценки R2 такова, что она очень хорошо оценивает степень обучения модели получая долю сумм отклонений предсказаний от сумм отклонений самих тестовых данных.

Оценка 

In [ ]:
def plot_test(X_train, X_test, y_train, y_test, y_pred):
  df_train = pd.DataFrame()
  df_test = pd.DataFrame()

  df_test['X_test'] = [s[0] for s in X_test.tolist()]
  df_test['y_test'] = y_test
  df_test['y_pred'] = y_pred

  df_train['X_train'] = [s[0] for s in X_train.tolist()]
  df_train['y_train'] = y_train


  fig = go.Figure(
    px.scatter(df_train, x='X_train', y='y_train', labels={'X_train': 'train'}).data +
    px.scatter(df_test, x='X_test', y='y_test', color_discrete_sequence=['red']).data +
    px.scatter(df_test, x='X_test', y='y_pred', color_discrete_sequence=['lime'], trendline="lowess", trendline_options=dict(frac=0.1)).data
  )

  fig.data[0].name="train"
  fig.data[1].name="test"
  fig.data[2].name="predictions"
  fig.update_traces(showlegend=True)


  return fig

In [ ]:
plot_test(X_train_1, X_test_1, y_train_1, y_test_1, y_pred_lr_1)

Точность довольно высокая, оно и видно на графике, красные точки тренировочных данных недалеко от линии регрессии. Шума не так много.

### Ridge регуляризация

Напишем функцию для вывода визуализации весов, они будут в том же порядке как и расположены фичи в датасете

In [ ]:
def plot_weights(coefs, columns=None, show_cols=False):
  df_plot = pd.DataFrame()

  df_plot['y'] = coefs.squeeze()
  if show_cols:
    df_plot['x'] = columns
  else:
    df_plot['x'] = ['w' + str(s) for s in range(len(coefs.squeeze()))]
  df_plot['c'] = df_plot.y.apply(lambda x: '>=0' if x >= 0 else '<0')


  return px.bar(df_plot, y='y', x='x', color='c', color_discrete_map={'>=0': 'blue', '<0': 'red'}).update_xaxes(categoryorder='array', categoryarray=df_plot['x'])

Регуляризация работает посредством уменьшения весов, для корректной работы их должно быть несколько, применим функцию для создания новых фич

In [ ]:
poly = PolynomialFeatures(degree=5, include_bias=False)

X_train_1_ = poly.fit_transform(X_train_1)
X_test_1_ = poly.fit_transform(X_test_1)

pd.DataFrame(X_train_1_).head()

,0,1,2,3,4
0,2.275023,5.175731,11.774910,26.788196,60.943773
1,3.812399,14.534388,55.410888,211.248426,805.363330
2,1.970867,3.884317,7.655473,15.087921,29.736287
3,-0.188665,0.035595,-0.006715,0.001267,-0.000239
4,0.656851,0.431453,0.283401,0.186152,0.122274


In [ ]:
model_r_1 = Ridge().fit(X_train_1_, y_train_1)

y_pred_1 = model_r_1.predict(X_test_1_)

print(f'R2 score: {r2_score(y_test_1, y_pred_1)}')
print(f'MAE: {mean_absolute_error(y_test_1, y_pred_1)}')
print(f'MSE: {mean_squared_error(y_test_1, y_pred_1)}')

R2 score: 0.9748219064003737
MAE: 0.10999609160916135
MSE: 0.013324545612539426


Метрика R2 score увеличилась, значит новые фичи помогли

In [ ]:
model_r_1 = Ridge(alpha=2).fit(X_train_1_, y_train_1)

y_pred_1 = model_r_1.predict(X_test_1_)

print(f'R2 score: {r2_score(y_test_1, y_pred_1)}')
print(f'MAE: {mean_absolute_error(y_test_1, y_pred_1)}')
print(f'MSE: {mean_squared_error(y_test_1, y_pred_1)}')

R2 score: 0.9751224330993771
MAE: 0.11034894331731011
MSE: 0.013165503320762625


С новым коэффициентом снижения весов мы получили ещё большую производительность, что показывает работу с весами в действии

In [ ]:
plot_weights(model_r_1.coef_)

Если поиграться с параметром, так называемого, сужения, то можно увидеть, что веса стремятся к 0, но не достигают его

### Lasso регуляризация

In [ ]:
model_l_1 = Lasso(tol=0.015).fit(X_train_1_, y_train_1)

y_pred_l_1 = model_l_1.predict(X_test_1_)

print(f'R2 score: {r2_score(y_test_1, y_pred_l_1)}')
print(f'MAE: {mean_absolute_error(y_test_1, y_pred_l_1)}')
print(f'MSE: {mean_squared_error(y_test_1, y_pred_l_1)}')

R2 score: 0.8825365749486707
MAE: 0.22505338308804612
MSE: 0.06216303703489199


Пришлось задать допустимое отклонение для оптимизации побольше, так как модель просто не достигала такого маленького, как дефолтное: 0.0001

In [ ]:
model_l_1 = Lasso(alpha=0.03, tol=0.015).fit(X_train_1_, y_train_1)

y_pred_1 = model_l_1.predict(X_test_1_)

print(f'R2 score: {r2_score(y_test_1, y_pred_1)}')
print(f'MAE: {mean_absolute_error(y_test_1, y_pred_1)}')
print(f'MSE: {mean_squared_error(y_test_1, y_pred_1)}')

R2 score: 0.9707146995443922
MAE: 0.12019617077835372
MSE: 0.015498128170572065


In [ ]:
plot_weights(model_l_1.coef_)

Здесь же видим другую ситуацию, веса легко становятся незначимыми, поэтому Lasso можно использовать и как feature selection

## 2 датасет

### Линейная регрессия

In [ ]:
model_lr_2 = LinearRegression().fit(X_train_2, y_train_2)

y_pred_lr_2 = model_lr_2.predict(X_test_2)

print(f'R2 score: {r2_score(y_test_2, y_pred_lr_2)}')
print(f'MAE: {mean_absolute_error(y_test_2, y_pred_lr_2)}')
print(f'MSE: {mean_squared_error(y_test_2, y_pred_lr_2)}')

R2 score: 0.4996928835134836
MAE: 18.471891475129304
MSE: 561.6546801154793


Видим очень плохие результаты, оно и понятно, сейчас посмотрим на графике

In [ ]:
plot_test(X_train_2, X_test_2, y_train_2, y_test_2, y_pred_lr_2)

Для того, чтобы линия была более кривой, нужно добавить полиномиальные фичи

In [ ]:
poly = PolynomialFeatures(degree=5, include_bias=False)

X_train_2_ = poly.fit_transform(X_train_2)
X_test_2_ = poly.fit_transform(X_test_2)

pd.DataFrame(X_train_2_).head()

,0,1,2,3,4
0,3.176902,10.092708,32.063545,101.862746,323.607980
1,3.696999,13.667803,50.529856,186.808835,690.632107
2,2.465781,6.080074,14.992127,36.967294,91.153233
3,2.405397,5.785934,13.917466,33.477027,80.525533
4,2.043799,4.177113,8.537179,17.448276,35.660766


In [ ]:
model_lr_2 = LinearRegression().fit(X_train_2_, y_train_2)

y_pred_2 = model_lr_2.predict(X_test_2_)

print(f'R2 score: {r2_score(y_test_2, y_pred_2)}')
print(f'MAE: {mean_absolute_error(y_test_2, y_pred_2)}')
print(f'MSE: {mean_squared_error(y_test_2, y_pred_2)}')

R2 score: 0.9891307401841827
MAE: 2.915147836068811
MSE: 12.202046390658223


In [ ]:
plot_test(X_train_2, X_test_2, y_train_2, y_test_2, y_pred_2)

In [ ]:
plot_weights(model_lr_2.coef_)

Как видно, сработала только одна дополнительная фича, но вполне неплохо

Теперь видим эффект приминения полиномиальных фич

### Ridge регуляризация

In [ ]:
model_r_2 = Ridge().fit(X_train_2_, y_train_2)

y_pred_r_2 = model_r_2.predict(X_test_2_)

print(f'R2 score: {r2_score(y_test_2, y_pred_r_2)}')
print(f'MAE: {mean_absolute_error(y_test_2, y_pred_r_2)}')
print(f'MSE: {mean_squared_error(y_test_2, y_pred_r_2)}')

R2 score: 0.9891722175812798
MAE: 2.9103499292634076
MSE: 12.155483043004411


In [ ]:
plot_test(X_train_2, X_test_2, y_train_2, y_test_2, y_pred_r_2)

Паттерн степенной функции повторяется

Видим, что тут дефолтный параметр альфа 1 сжатия, сделал всё прекрасно, лучше чем обычная регрессия, посмотрим на веса

In [ ]:
model_r_2.coef_

array([[ 1.38557891e+00, -2.04710021e+00,  4.66150760e-01,
         8.51008144e-03, -4.86922544e-04]])

In [ ]:
plot_weights(model_r_2.coef_)

Как видно чуть выше, веса не нулевые  и отличаются от обычной регресии, чего мы и добивались

### Lasso регуляризация

In [ ]:
model_l_2 = Lasso().fit(X_train_2_, y_train_2)

y_pred_l_2 = model_l_2.predict(X_test_2_)

print(f'R2 score: {r2_score(y_test_2, y_pred_l_2)}')
print(f'MAE: {mean_absolute_error(y_test_2, y_pred_l_2)}')
print(f'MSE: {mean_squared_error(y_test_2, y_pred_l_2)}')

R2 score: 0.9896131429184093
MAE: 2.708945597678861
MSE: 11.660491524755765


Хорошие результаты лучше чем у Ridge, посмотрим на график

In [ ]:
plot_test(X_train_2, X_test_2, y_train_2, y_test_2, y_pred_l_2)

Также видим практически точное попадание в тестовые точки

In [ ]:
model_l_2.coef_

array([ 5.28206803e-02, -1.60764420e+00,  5.22395773e-01, -1.52169847e-02,
        1.02003394e-03])

In [ ]:
plot_weights(model_l_2.coef_)

Для таких данных на уровне сжатия 1 не должно быть ухода какого-нибудь веса в 0, так что тут всё логично

## 3 датасет

### Линейная регрессия

In [ ]:
model_lr_3 = LinearRegression().fit(X_train_3, y_train_3)

y_pred_lr_3 = model_lr_3.predict(X_test_3)

print(f'R2 score: {r2_score(y_test_3, y_pred_lr_3)}')
print(f'MAE: {mean_absolute_error(y_test_3, y_pred_lr_3)}')
print(f'MSE: {mean_squared_error(y_test_3, y_pred_lr_3)}')

R2 score: 0.8827425109075514
MAE: 292.6415647269976
MSE: 133174.05574811486


Неплохая точность, но это обычная линейная регрессия, посмотрим на график

In [ ]:
plot_test(X_train_3, X_test_3, y_train_3, y_test_3, y_pred_lr_3)

Видим попадания и большую точность исключительно потому, что линия касается изначальной функции до применения шума во многих местах, сделаем полиномиальную регрессию. 

In [ ]:
poly = PolynomialFeatures(degree=5, include_bias=False)

X_train_3_ = poly.fit_transform(X_train_3)
X_test_3_ = poly.fit_transform(X_test_3)

pd.DataFrame(X_train_3_).head()

,0,1,2,3,4
0,5.880648,34.582018,203.364668,1195.915978,7.032761e+03
1,0.035474,0.001258,0.000045,0.000002,5.617679e-08
2,-9.655572,93.230065,-900.189582,8691.845072,-8.392473e+04
3,2.170322,4.710299,10.222866,22.186913,4.815275e+01
4,5.321663,28.320092,150.709974,802.027624,4.268120e+03


In [ ]:
model_lr_3 = LinearRegression().fit(X_train_3_, y_train_3)

y_pred_lr_3 = model_lr_3.predict(X_test_3_)

print(f'R2 score: {r2_score(y_test_3, y_pred_lr_3)}')
print(f'MAE: {mean_absolute_error(y_test_3, y_pred_lr_3)}')
print(f'MSE: {mean_squared_error(y_test_3, y_pred_lr_3)}')

R2 score: 0.9917139298963843
MAE: 77.92322949805616
MSE: 9410.823738871719


In [ ]:
plot_test(X_train_3, X_test_3, y_train_3, y_test_3, y_pred_lr_3)

Другое дело, прослеживается практически точное попадание

In [ ]:
plot_weights(model_lr_3.coef_)

Если приблизить два последних веса на графике, то будет видно, что они не нулевые, а главный вес у одной фичи, так как она описывает функцию, которая была в основе генерации данных

### Ridge регуляризация

In [ ]:
model_r_3 = Ridge().fit(X_train_3_, y_train_3)

y_pred_r_3 = model_r_3.predict(X_test_3_)

print(f'R2 score: {r2_score(y_test_3, y_pred_r_3)}')
print(f'MAE: {mean_absolute_error(y_test_3, y_pred_r_3)}')
print(f'MSE: {mean_squared_error(y_test_3, y_pred_r_3)}')

R2 score: 0.9917140092702561
MAE: 77.9230774297794
MSE: 9410.733590766835


Видим, что совсем чутка лучше, попробуем задать параметр alpha побольше

In [ ]:
model_r_3 = Ridge(alpha=5).fit(X_train_3_, y_train_3)

y_pred_r_3 = model_r_3.predict(X_test_3_)

print(f'R2 score: {r2_score(y_test_3, y_pred_r_3)}')
print(f'MAE: {mean_absolute_error(y_test_3, y_pred_r_3)}')
print(f'MSE: {mean_squared_error(y_test_3, y_pred_r_3)}')

R2 score: 0.9917143204511454
MAE: 77.92248565196299
MSE: 9410.380170090486


Другое дело, посмотрим на график и веса

In [ ]:
plot_test(X_train_3, X_test_3, y_train_3, y_test_3, y_pred_r_3)

Тут как видно, попадания прям замечательные

In [ ]:
plot_weights(model_r_3.coef_)

Вероятно регуляризация уже больше приближает к нулю те два последних веса и первые два, по идее с хорошим альфа у лассо, мы можем обнулить эти веса

### Lasso регуляризация

In [ ]:
model_l_3 = Lasso().fit(X_train_3_, y_train_3)

y_pred_l_3 = model_l_3.predict(X_test_3_)

print(f'R2 score: {r2_score(y_test_3, y_pred_l_3)}')
print(f'MAE: {mean_absolute_error(y_test_3, y_pred_l_3)}')
print(f'MSE: {mean_squared_error(y_test_3, y_pred_l_3)}')

R2 score: 0.9917203603693212
MAE: 77.91904899944537
MSE: 9403.520391614238


Как и ожидалось, обнуление дало о себе знать, точность поднялась на одну десятитысячную, вместо статысячной как на Ridge

In [ ]:
plot_test(X_train_3, X_test_3, y_train_3, y_test_3, y_pred_l_3)

Ещё ближе попадания

Но стоит сделать большое alpha, чтобы увидеть прям практически нулевые веса

In [ ]:
model_l_3 = Lasso(alpha=4).fit(X_train_3_, y_train_3)

y_pred_l_3 = model_l_3.predict(X_test_3_)

print(f'R2 score: {r2_score(y_test_3, y_pred_l_3)}')
print(f'MAE: {mean_absolute_error(y_test_3, y_pred_l_3)}')
print(f'MSE: {mean_squared_error(y_test_3, y_pred_l_3)}')

R2 score: 0.9917272010092053
MAE: 77.9325770361694
MSE: 9395.751201225368


In [ ]:
model_l_3.coef_

array([-0.00000000e+00, -1.27558752e+00,  2.55121028e+00,  5.69303487e-03,
       -9.48703953e-04])

In [ ]:
plot_weights(model_l_3.coef_)

Что и ожидалось, первый вес улетел в ноль

# 2.  Для набора данных Cars Moldova

Не очень логично делать задание конкретно по пунктам, разделение пунктов будет по датасетам, там же сможем и посмотреть производительность без категориальных признаков.

## Загрузка всех датасетов с разной предобработкой данных

Все данные у меня лежат тамже, где и анаконда, так что подгрузка просто по имени файла

In [ ]:
df_st_norm = pd.read_csv('cars_moldova_norm_and_stand.csv').drop(columns=['Unnamed: 0'])
df_st_norm

,Year,Distance,Engine_capacity(cm3),Price(euro),year_distance,Transmission_Automatic,Transmission_Manual,Model_1 Series,Model_100,Model_1007,...,distance_type_average,distance_type_high,distance_type_optimal,distance_type_very_high,Make_rarity_common,Make_rarity_rare,Price_cat_average,Price_cat_cheap,Year_category_old,Year_category_regular
0,0.419955,0.285702,0.360288,7700.0,0.481595,1,0,0,0,0,...,0,1,0,0,0,1,0,1,0,1
1,0.820193,-0.274774,0.300240,8500.0,0.384476,0,1,0,0,0,...,1,0,0,0,0,1,0,1,0,1
2,-1.314409,-1.535836,0.280224,2200.0,-1.538493,0,1,0,0,0,...,0,0,1,0,0,1,1,0,1,0
3,0.553368,-0.508306,0.300240,6500.0,-0.285004,0,1,0,0,0,...,1,0,0,0,0,1,0,1,0,1
4,-0.247108,0.332408,0.320256,4100.0,-0.114073,0,1,0,0,0,...,0,1,0,0,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35964,-0.780758,0.565940,0.360288,4400.0,-0.256515,0,1,0,0,0,...,0,1,0,0,1,0,1,0,1,0
35965,0.953606,-0.704473,0.300240,7000.0,-0.089654,0,1,0,0,0,...,0,0,1,0,0,1,0,1,0,1
35966,0.153130,-1.533744,0.300240,4500.0,-1.536525,0,1,0,0,0,...,0,0,1,0,0,1,1,0,1,0
35967,-0.247108,1.920424,0.400320,4000.0,1.096688,0,1,0,0,0,...,0,0,0,1,0,1,1,0,1,0


In [ ]:
df_st = pd.read_csv('cars_moldova_stand.csv').drop(columns=['Unnamed: 0'])
df_st

,Year,Distance,Engine_capacity(cm3),Price(euro),year_distance,Transmission_Automatic,Transmission_Manual,Model_1 Series,Model_100,Model_1007,...,distance_type_average,distance_type_high,distance_type_optimal,distance_type_very_high,Make_rarity_common,Make_rarity_rare,Price_cat_average,Price_cat_cheap,Year_category_old,Year_category_regular
0,0.419955,0.285702,-0.046315,7700.0,0.481595,1,0,0,0,0,...,0,1,0,0,0,1,0,1,0,1
1,0.820193,-0.274774,-0.525630,8500.0,0.384476,0,1,0,0,0,...,1,0,0,0,0,1,0,1,0,1
2,-1.314409,-1.535836,-0.685401,2200.0,-1.538493,0,1,0,0,0,...,0,0,1,0,0,1,1,0,1,0
3,0.553368,-0.508306,-0.525630,6500.0,-0.285004,0,1,0,0,0,...,1,0,0,0,0,1,0,1,0,1
4,-0.247108,0.332408,-0.365858,4100.0,-0.114073,0,1,0,0,0,...,0,1,0,0,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35964,-0.780758,0.565940,-0.046315,4400.0,-0.256515,0,1,0,0,0,...,0,1,0,0,1,0,1,0,1,0
35965,0.953606,-0.704473,-0.525630,7000.0,-0.089654,0,1,0,0,0,...,0,0,1,0,0,1,0,1,0,1
35966,0.153130,-1.533744,-0.525630,4500.0,-1.536525,0,1,0,0,0,...,0,0,1,0,0,1,1,0,1,0
35967,-0.247108,1.920424,0.273228,4000.0,1.096688,0,1,0,0,0,...,0,0,0,1,0,1,1,0,1,0


In [ ]:
df_norm = pd.read_csv('cars_moldova_norm.csv').drop(columns=['Unnamed: 0'])
df_norm

,Year,Distance,Engine_capacity(cm3),Price(euro),year_distance,Transmission_Automatic,Transmission_Manual,Model_1 Series,Model_100,Model_1007,...,distance_type_average,distance_type_high,distance_type_optimal,distance_type_very_high,Make_rarity_common,Make_rarity_rare,Price_cat_average,Price_cat_cheap,Year_category_old,Year_category_regular
0,0.836066,0.195000,0.360288,7700.0,6.331169e-02,1,0,0,0,0,...,0,1,0,0,0,1,0,1,0,1
1,0.885246,0.135000,0.300240,8500.0,6.026786e-02,0,1,0,0,0,...,1,0,0,0,0,1,0,1,0,1
2,0.622951,0.000001,0.280224,2200.0,1.488095e-07,0,1,0,0,0,...,0,0,1,0,0,1,1,0,1,0
3,0.852459,0.110000,0.300240,6500.0,3.928571e-02,0,1,0,0,0,...,1,0,0,0,0,1,0,1,0,1
4,0.754098,0.200000,0.320256,4100.0,4.464286e-02,0,1,0,0,0,...,0,1,0,0,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35964,0.688525,0.225000,0.360288,4400.0,4.017857e-02,0,1,0,0,0,...,0,1,0,0,1,0,1,0,1,0
35965,0.901639,0.089000,0.300240,7000.0,4.540816e-02,0,1,0,0,0,...,0,0,1,0,0,1,0,1,0,1
35966,0.803279,0.000225,0.300240,4500.0,6.181319e-05,0,1,0,0,0,...,0,0,1,0,0,1,1,0,1,0
35967,0.754098,0.370000,0.400320,4000.0,8.258929e-02,0,1,0,0,0,...,0,0,0,1,0,1,1,0,1,0


## Нормализированные данные

In [ ]:
X = df_norm.loc[:, df_norm.columns != 'Price(euro)']
y = df_norm[['Price(euro)']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

Получили такую ошибку: 


```
The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.
```



In [ ]:
# df_norm['Price(euro)'].apply(lambda x: x//100*100).value_counts()[df_norm['Price(euro)'].apply(lambda x: x//100*100).value_counts() == 1]

Данное количество данных можно удалять, чтобы регрессия работала. Но перенесём это в первое задание, чтобы здесь посреди применения модели этим не заниматься


### Lasso

In [ ]:
moldova_l = Lasso().fit(X_train, y_train)

y_pred = moldova_l.predict(X_test)

print(f'R2 score: {r2_score(y_test, y_pred)}')
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'RMSE: {mean_squared_error(y_test, y_pred, squared=False)}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')

R2 score: 0.7375395040206502
MAE: 2520.789955492361
RMSE: 3904.8998459943277
MAPE: 0.49264329244904415


Модель приемлимо отработала, но хотелось бы лучше, средняя ошибка большая

In [ ]:
moldova_l = Lasso(alpha=0.7).fit(X_train, y_train)

y_pred = moldova_l.predict(X_test)

print(f'R2 score: {r2_score(y_test, y_pred)}')
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'RMSE: {mean_squared_error(y_test, y_pred, squared=False)}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')

R2 score: 0.741448600331285
MAE: 2502.3942339511314
RMSE: 3875.710891731895
MAPE: 0.4953119425369495


In [ ]:
plot_weights(moldova_l.coef_, X.columns, show_cols=True)

Если приблизить, то можем видеть, каким признакам отдаётся предпочтение

### Ridge

Может ridge сработает лучше, так как веса не будут занижены до нуля, ведь в таких данных нет конкретной зависимости, которую можно смело выделять, убирая остальные признаки

In [ ]:
moldova_r = Ridge().fit(X_train, y_train)

y_pred = moldova_r.predict(X_test)

print(f'R2 score: {r2_score(y_test, y_pred)}')
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'RMSE: {mean_squared_error(y_test, y_pred, squared=False)}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')

R2 score: 0.7483983811643856
MAE: 2465.4332975327684
RMSE: 3823.267124256282
MAPE: 0.5081944796993725


In [ ]:
moldova_r = Ridge(alpha=6).fit(X_train, y_train)

y_pred = moldova_r.predict(X_test)

print(f'R2 score: {r2_score(y_test, y_pred)}')
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'RMSE: {mean_squared_error(y_test, y_pred, squared=False)}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')

R2 score: 0.7428159826166114
MAE: 2494.6358049703244
RMSE: 3865.448708103546
MAPE: 0.492668273735544


Да, мы получили результаты лучше, но совсем немного, посмотрим на веса

In [ ]:
plot_weights(moldova_r.coef_, X.columns, show_cols=True)

Всё-таки некоторые модели машин могут влиять на результаты

### Данные без категорий

In [ ]:
df_norm

,Year,Distance,Engine_capacity(cm3),Price(euro),year_distance,Transmission_Automatic,Transmission_Manual,Model_1 Series,Model_100,Model_1007,...,distance_type_average,distance_type_high,distance_type_optimal,distance_type_very_high,Make_rarity_common,Make_rarity_rare,Price_cat_average,Price_cat_cheap,Year_category_old,Year_category_regular
0,0.836066,0.195000,0.360288,7700.0,6.331169e-02,1,0,0,0,0,...,0,1,0,0,0,1,0,1,0,1
1,0.885246,0.135000,0.300240,8500.0,6.026786e-02,0,1,0,0,0,...,1,0,0,0,0,1,0,1,0,1
2,0.622951,0.000001,0.280224,2200.0,1.488095e-07,0,1,0,0,0,...,0,0,1,0,0,1,1,0,1,0
3,0.852459,0.110000,0.300240,6500.0,3.928571e-02,0,1,0,0,0,...,1,0,0,0,0,1,0,1,0,1
4,0.754098,0.200000,0.320256,4100.0,4.464286e-02,0,1,0,0,0,...,0,1,0,0,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35964,0.688525,0.225000,0.360288,4400.0,4.017857e-02,0,1,0,0,0,...,0,1,0,0,1,0,1,0,1,0
35965,0.901639,0.089000,0.300240,7000.0,4.540816e-02,0,1,0,0,0,...,0,0,1,0,0,1,0,1,0,1
35966,0.803279,0.000225,0.300240,4500.0,6.181319e-05,0,1,0,0,0,...,0,0,1,0,0,1,1,0,1,0
35967,0.754098,0.370000,0.400320,4000.0,8.258929e-02,0,1,0,0,0,...,0,0,0,1,0,1,1,0,1,0


In [ ]:
X = df_norm.loc[:, df_norm.columns != 'Price(euro)']
X = X.loc[:, X.columns[:4]]
y = df_norm[['Price(euro)']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [ ]:
moldova_r = Ridge(alpha=6).fit(X_train, y_train)

y_pred = moldova_r.predict(X_test)

print(f'R2 score: {r2_score(y_test, y_pred)}')
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'RMSE: {mean_squared_error(y_test, y_pred, squared=False)}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')

R2 score: 0.49064527714790784
MAE: 3675.464873786352
RMSE: 5439.864668164901
MAPE: 0.8380758533092567


Всё ооочень плохо, тут даже исправлять нечего, при том, что нормализация это самый универсальный вариант

## Стандартизированные данные

### Lasso

In [ ]:
X = df_st.loc[:, df_st.columns != 'Price(euro)']
y = df_st[['Price(euro)']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [ ]:
moldova_l = Lasso().fit(X_train, y_train)

y_pred = moldova_l.predict(X_test)

print(f'R2 score: {r2_score(y_test, y_pred)}')
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'RMSE: {mean_squared_error(y_test, y_pred, squared=False)}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')

R2 score: 0.7369409292655442
MAE: 2514.9635000342932
RMSE: 3909.3501220394887
MAPE: 0.49589386928078705


Результаты +- такие же попробуем поменять альфа

In [ ]:
moldova_l = Lasso(alpha=0.4).fit(X_train, y_train)

y_pred = moldova_l.predict(X_test)

print(f'R2 score: {r2_score(y_test, y_pred)}')
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'RMSE: {mean_squared_error(y_test, y_pred, squared=False)}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')

R2 score: 0.7433747522628995
MAE: 2476.3173954166195
RMSE: 3861.2473003388145
MAPE: 0.49683737766357355


Такая же история в целом, посмотрим на веса

In [ ]:
plot_weights(moldova_l.coef_, X.columns, show_cols=True)

Интересно, здесь уже модели преобладают в весах, с точки зрения логики, это очень плохо, так как модели постоянно меняются, нужно больше основываться на характеристиках, а у них тут совсем небольшие веса

### Ridge

In [ ]:
moldova_r = Ridge().fit(X_train, y_train)

y_pred = moldova_r.predict(X_test)

print(f'R2 score: {r2_score(y_test, y_pred)}')
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'RMSE: {mean_squared_error(y_test, y_pred, squared=False)}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')

R2 score: 0.746098915184634
MAE: 2467.8001942725414
RMSE: 3840.698405081527
MAPE: 0.5073894520698926


Результаты с точки зрения ошибки похуже, а с точки зрения степени обучения лучше

In [ ]:
moldova_r = Ridge(alpha=6).fit(X_train, y_train)

y_pred = moldova_r.predict(X_test)

print(f'R2 score: {r2_score(y_test, y_pred)}')
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'RMSE: {mean_squared_error(y_test, y_pred, squared=False)}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')

R2 score: 0.7422788461802391
MAE: 2494.982284097161
RMSE: 3869.4831547347435
MAPE: 0.50068662424421


Нормализированные данные дают лучше резултаты

In [ ]:
plot_weights(moldova_r.coef_, X.columns, show_cols=True)

Такая же ситуация с весами, предпочтение моделям автомобилей

Смысла делать прогноз на данных без категорий не вижу, так как здесь основные веса отданы им

## Стандартизированные и нормализированные данные

In [ ]:
X = df_st_norm.loc[:, df_st_norm.columns != 'Price(euro)']
y = df_st_norm[['Price(euro)']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

### Lasso

In [ ]:
moldova_l = Lasso().fit(X_train, y_train)

y_pred = moldova_l.predict(X_test)

print(f'R2 score: {r2_score(y_test, y_pred)}')
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'RMSE: {mean_squared_error(y_test, y_pred, squared=False)}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')

R2 score: 0.7425778973296058
MAE: 2458.8313685175253
RMSE: 3867.2374926128987
MAPE: 0.48384650241319077


Как я и предполагал в первом задании данная техника предобработки отработает лучше всего, сейчас поиграем с параметром альфа и можно будет подводить итоги

In [ ]:
moldova_l = Lasso(alpha=0.3).fit(X_train, y_train)

y_pred = moldova_l.predict(X_test)

print(f'R2 score: {r2_score(y_test, y_pred)}')
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'RMSE: {mean_squared_error(y_test, y_pred, squared=False)}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')

Посмотрим веса, опять же на plotly очень удобно смотреть, так как можно приблизить конкретное место

In [ ]:
plot_weights(moldova_l.coef_, X.columns, show_cols=True)

Не очень хорошая ситуация с весами, предпочтение отдаётся моделям

### Ridge

In [ ]:
moldova_r = Ridge().fit(X_train, y_train)

y_pred = moldova_r.predict(X_test)

print(f'R2 score: {r2_score(y_test, y_pred)}')
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'RMSE: {mean_squared_error(y_test, y_pred, squared=False)}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')

Результат хуже чем у Lasso по точности, но лучше по степени обучения, посмотрим сможет ли параметр альфа исправить это

In [ ]:
moldova_r = Ridge(alpha=5).fit(X_train, y_train)

y_pred = moldova_r.predict(X_test)

print(f'R2 score: {r2_score(y_test, y_pred)}')
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'RMSE: {mean_squared_error(y_test, y_pred, squared=False)}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')

Всё-таки Lasso лучше

In [ ]:
plot_weights(moldova_r.coef_, X.columns, show_cols=True)

Такая же история

# Вывод

Подытожим, нам удалось применить регуляризацию на данных о продажах машин в молдавии.

В итоге лучшими оказазались:
- Ridge на нормализированных данных, где хорошие веса были у главных характеристик автомобилей, особенно год производства
- Lasso на нормализированных и стандартизированных данных, веса уже не такие хорошие, характеристикам автомобиля не даётся весомого предпочтения.

Стоит использовать Lasso на нормализированных и стандартизированных данных, но иметь в виду, если точность будет падать, перейти на Ridge на нормализированных данных.